In [63]:
#import bin-encoding data
import pandas as pd
import numpy as np
import scipy.stats as stats
import sklearn
import random
import os
from pathlib import Path
from sklearn.linear_model import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report 
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances, manhattan_distances, euclidean_distances

In [48]:
def Cosinepairup(npV, rows, Y):
    for i in range(rows):
        #find most related articles indexed
        a = npV[i,:]
        index = np.argpartition(a, -4)[-4:]
        index2= index[np.argsort(a[index])]

        #show the index in X matrix
        #print(i)
        #print(index)
        #print(index2)
        #show the similarity value
        #print(a[index2])

        related = []
        #ensure that same article is not ranked as the most similar article
        for j in range(3,-1,-1):
            if i == index2[j]:
                pass #do not count the same article as most related
            elif len(related) == 3:
                pass
            else:
                related.append(str(index2[j]))

        Y.at[i, 'related_articles'] = ', '.join(related)

    return Y[['related_articles']]

In [1]:
def recommender(Encoding, contextTable):
    #Encoded = Encoding.drop(columns=['article_id'])
    
    #Similarity matrix between each article
    Csim = cosine_similarity(Encoded)

    #convert to numpy
    npV = np.asarray(Csim)
    rows = np.size(npV,0)

    
    #match most related articles by article index
    finalMatches = Cosinepairup(npV, rows, Encoded)
    finalTable = contextTable.join(finalMatches, how='left')
    
    return finalTable
    

## Code below to compare cosine with other similarity metrics

In [67]:
#Similarity matrix between each article
temp = pd.read_csv("matrixtemp.csv")
Encoded = temp.drop(columns=['article_id'])
Csim = cosine_similarity(Encoded)
ManhSim = manhattan_distances(Encoded)
EuclSim = euclidean_distances(Encoded)

#convert to numpy
npV = np.asarray(Csim)
rowsC = np.size(npV,0)
npEuc = np.asarray(EuclSim)
rows = np.size(npEuc,0)
npMan = np.asarray(ManhSim)
rowsMan = np.size(npMan,0)
print(npMan)

[[  0.  28.  30. ... 157. 162. 161.]
 [ 28.   0.  10. ... 143. 144. 147.]
 [ 30.  10.   0. ... 139. 140. 147.]
 ...
 [157. 143. 139. ...   0.  13.  18.]
 [162. 144. 140. ...  13.   0.  11.]
 [161. 147. 147. ...  18.  11.   0.]]


In [68]:
print(npV)

[[1.         0.14433757 0.         ... 0.00311585 0.         0.00613699]
 [0.14433757 1.         0.375      ... 0.3534916  0.35349253 0.35874832]
 [0.         0.375      1.         ... 0.7069832  0.70698507 0.70686706]
 ...
 [0.00311585 0.3534916  0.7069832  ... 1.         0.99965313 0.99954363]
 [0.         0.35349253 0.70698507 ... 0.99965313 1.         0.99971663]
 [0.00613699 0.35874832 0.70686706 ... 0.99954363 0.99971663 1.        ]]


In [69]:
def pairup(npV, rows, Y):
    for i in range(rows):
        #find most related articles indexed (smallest distance)
        a = npV[i,:]
        index2= np.argsort(a)

        #show the index 
        #print(i)
        #print(index)
        print(a)
        print(index2)
        #show the similarity value
        #print(a[index2])

        related = []
        #ensure that same article is not ranked as the most similar article
        for j in range(4):
            if i == index2[j]:
                pass #do not count the same article as most related
            elif len(related) == 3:
                pass
            else:
                related.append(str(index2[j]))

        Y.at[i, 'related_articles'] = ', '.join(related)

    return Y[['related_articles']]

In [70]:
pairup(npEuc, rows, Encoded)

[  0.           5.29150262   5.65685425   3.           6.164414
   7.           8.06225775   8.88819442   9.59166305  10.29563014
  11.13552873  12.40967365  13.03840481  13.89244399  14.93318452
  15.          16.88194302  17.74823935  19.18332609  19.74841766
  20.78460969  21.81742423  22.60530911  23.66431913  24.51530134
  25.47547841  26.5894716   27.49545417  28.47806173  29.46183973
  30.4466747   31.52776554  32.49615362  33.33166662  34.36568055
  35.34119409  36.31803959  37.37646318  38.37968212  39.42080669
  40.29888336  41.34005322  42.3083916   43.32435804  44.31703961
  45.38722287  46.28174586  47.33920151  48.33218389  49.29503018
  50.26927491  51.23475383  52.20153254  53.30103188  54.24942396
  55.29014379  56.26721959  57.20139858  58.25804665  59.22837158
  60.23288139  61.24540799  62.19324722  63.2455532   64.18722614
  65.21502894  66.19667665  67.19374971  68.1762422   69.23149572
  70.17121917  71.18286311  72.1734023   73.19152956  74.20916385
  75.2196782

 36.12478374 37.20215048]
[ 96  95  94  93  98  99  97 100  92 101  91  90  89 102 103 104  88  87
 105 106  86 107  85 108  84  83 109  82 110 111  81 112  80  79 113  78
 114  77 115  76 116  75 117  74 118 119  73  72 120  71 121 122  70 123
  69 124  68 125  67  66 126  65 127  64 128  63 129 130  62 131  61 132
  60  59 133  58  57  56  55  54  53  52  51  50  49  48  47  46  45  44
  43  42  41  40  39  38  37  36  35  34  33  32  31  30  29  28  27  26
  25  24  23  22  21  20  19  18  17  16  15  14  13  12  11  10   9   8
   7   6   5   4   3   2   1   0]
[97.15966241 96.11451503 95.09994742 94.16474924 93.12357381 92.09234496
 91.12079894 90.1276872  89.08422981 88.09086218 87.10912696 86.13942187
 85.11169132 84.11301921 83.09632964 82.18880702 81.09870529 80.09369513
 79.24645102 78.12809994 77.17512553 76.18398782 75.13321503 74.16198487
 73.15052973 72.09022125 71.19691005 70.14271167 69.15923655 68.13222439
 67.16397844 66.21178143 65.1459899  64.16385275 63.12685641 62.

,related_articles
0,"3, 1, 2"
1,"2, 5, 4"
2,"1, 5, 4"
3,"0, 4, 5"
4,"5, 2, 6"
5,"4, 6, 2"
6,"5, 8, 7"
7,"8, 5, 6"
8,"9, 10, 7"
9,"8, 10, 12"


In [71]:
pairup(npMan, rows, Encoded)

[  0.  28.  30.   3.  26.  29.  35.  37.  36.  34.  34.  44.  38.  37.
  41.  15.  45.  43.  62.  48.  52.  56.  49.  54.  49.  49.  57.  54.
  55.  56.  57.  64.  64.  55.  59.  59.  59.  65.  67.  72.  64.  69.
  68.  71.  72.  80.  72.  79.  80.  78.  77.  75.  73.  85.  81.  87.
  86.  80.  88.  86.  88.  91.  86.  94.  88.  93.  92.  93.  92. 101.
  94.  97.  97. 101. 105. 108. 100. 104. 102. 106. 110. 115. 108. 108.
 128. 112. 111. 110. 114. 117. 118. 119. 113. 116. 123. 123. 121. 128.
 127. 130. 126. 128. 142. 129. 128. 134. 131. 131. 138. 134. 132. 138.
 137. 142. 141. 140. 142. 162. 158. 144. 147. 146. 147. 152. 150. 152.
 152. 190. 149. 166. 155. 157. 162. 161.]
[  0   3  15   4   1   5   2   9  10   6   8   7  13  12  14  17  11  16
  19  24  22  25  20  23  27  28  33  29  21  26  30  34  36  35  18  40
  32  31  37  38  42  41  43  39  44  46  52  51  50  49  47  45  57  48
  54  53  56  59  62  55  58  60  64  61  68  66  67  65  63  70  71  72
  76  69  73  78  77  74  7

[105.  91.  91. 102.  91.  84.  88.  90.  83.  81.  83.  87.  81.  80.
  76.  90.  76.  74.  97.  75.  81.  83.  68.  71.  72.  64.  76.  69.
  64.  65.  64.  67.  65.  62.  56.  58.  56.  58.  64.  55.  49.  56.
  51.  52.  49.  49.  47.  50.  47.  45.  40.  38.  38.  40.  36.  42.
  43.  35.  33.  33.  35.  36.  27.  31.  25.  28.  25.  26.  21.  28.
  25.  26.  16.  24.   0.  29.  17.  25.  19.  23.  23.  32.  27.  27.
  39.  37.  32.  29.  33.  38.  35.  36.  36.  37.  40.  38.  40.  49.
  40.  47.  45.  45.  57.  52.  49.  51.  48.  46.  53.  55.  51.  57.
  54.  59.  58.  69.  57.  81.  69.  61.  70.  67.  64.  69.  65.  67.
  75. 113.  70.  85.  76.  72.  77.  80.]
[ 74  72  76  78  68  79  80  73  64  70  77  66  71  67  83  82  62  69
  65  75  87  63  86  81  88  59  58  60  57  90  92  91  54  61  85  93
  52  95  51  89  84  96  94  53  98  50  55  56  49 101 100 107  48  99
  46 106  40  45  44  97 104  47 110  42 105  43 103 108 112 109  39  34
  41  36 116 102 111 114  3

,related_articles
0,"3, 15, 4"
1,"2, 5, 4"
2,"5, 1, 8"
3,"0, 15, 4"
4,"5, 2, 6"
5,"2, 8, 9"
6,"5, 8, 9"
7,"8, 5, 10"
8,"9, 5, 10"
9,"8, 10, 14"


In [72]:
Cosinepairup(npV, rowsC, Encoded)

,related_articles
0,"3, 15, 4"
1,"6, 5, 28"
2,"47, 24, 41"
3,"0, 15, 4"
4,"15, 19, 22"
5,"52, 102, 35"
6,"24, 28, 17"
7,"55, 16, 85"
8,"53, 42, 32"
9,"53, 14, 75"
